In [1]:
import torch
import torch.nn as nn

import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.tree import DecisionTreeClassifier


In [2]:
torch.cuda.is_available()

True

In [3]:
xlf = DecisionTreeClassifier(random_state=0)

In [4]:
enr_vec = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/enrollled/enroll/spkvecs.npy')
enr_len = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/enrollled/enroll/veclabs.npy')


In [5]:
enr_cos_sim = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/enrollled/test/result.npy')
not_cls_sim = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/notenrollled/result.npy')

enr_cls = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/enrollled/test/answer.npy')
not_cls = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/notenrollled/answer.npy')+51

In [9]:
enr_cos_t = enr_vec.transpose()
print(enr_cos_t.shape)
avg_enr_vec = []
enr_lab = []
for i in range(1, len(enr_len)):
    avg_enr_vec.append(enr_cos_t[enr_len[i-1]:enr_len[i],:].mean(axis=0).reshape(1,-1))
    
    for x in range(enr_len[i]-enr_len[i-1]):
        
        enr_lab.append(i)


(1392, 128)


In [11]:
avg_center = np.concatenate(avg_enr_vec, axis=0).transpose()
print(avg_center.shape)
print(enr_cos_t.shape, enr_lab.__len__())

(128, 50)
(1392, 128) 1392


In [12]:
print(not_cls_sim.shape)
enr_cos = np.matmul(enr_cos_t, avg_center)


x = np.append(enr_cos, not_cls_sim[:100, :], axis=0)
y = np.append(enr_lab, not_cls[:100])
xlf.fit(x, y)

(11699, 50)


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [13]:
pre_x = np.append(enr_cos_sim, not_cls_sim[100:], axis=0)
print(pre_x.shape)

(43086, 50)


In [14]:
real_y = np.append(enr_cls, not_cls[100:])
print(y.shape)


(1492,)


In [15]:
pred_y = xlf.predict(pre_x)

In [16]:
correct = 0
for m,n in zip(pred_y, real_y):
    if m==n:
        correct +=1
acc = correct / len(real_y)
print(acc) #20.192173791950982%

0.20192173791950982


In [48]:
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR

linearpred = nn.Linear(50, 51) # 使用cos分数预测

optimizer = optim.SGD(linearpred.parameters() ,lr=0.1,
                momentum=0.9,
                dampening=0,
                weight_decay=0.005)

scheduler = MultiStepLR(optimizer, milestones=[20, 20, 20], gamma=0.1)

output_softmax = nn.Softmax(dim=1)

crossentropy = nn.CrossEntropyLoss()

In [49]:
epoch = 80
t_x = torch.tensor(x).cuda()
print(t_x.shape)
linearpred = linearpred.cuda()
t_y = torch.tensor(y).long().cuda()
print(t_y.shape)

for i in range(epoch):
    
    p_y = linearpred(t_x)
    loss = crossentropy(p_y,t_y)
    
    p_y_lab = output_softmax(p_y)
    pred_one_labels = torch.max(p_y_lab, dim=1)[1]
    corrects = float((pred_one_labels.cuda() == t_y).sum().item())
    accuracy = corrects / len(y)
    
    if (i+1) % 5 == 0:
        print("Accuracy is %.4f %% " % (accuracy*100.)) # 100%
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

torch.Size([1492, 50])
torch.Size([1492])
Accuracy is 89.2761 % 
Accuracy is 92.4263 % 
Accuracy is 92.7614 % 
Accuracy is 93.1635 % 
Accuracy is 93.2306 % 
Accuracy is 93.4987 % 
Accuracy is 95.1072 % 
Accuracy is 99.1287 % 
Accuracy is 99.9330 % 
Accuracy is 100.0000 % 
Accuracy is 99.9330 % 
Accuracy is 99.9330 % 
Accuracy is 100.0000 % 
Accuracy is 100.0000 % 
Accuracy is 100.0000 % 
Accuracy is 100.0000 % 


In [50]:
lin_pre_x = torch.tensor(pre_x).cuda(0)
lin_pre_y = linearpred(lin_pre_x)
lin_p_y_lab = output_softmax(lin_pre_y)
lin_pred_labels = torch.max(lin_p_y_lab, dim=1)[1]

lin_real_y = torch.tensor(real_y).long().cuda()

corrects = float((lin_pred_labels.cuda() == lin_real_y).sum().item())
lin_test_accuracy = corrects / len(real_y)
print(lin_test_accuracy*100.) # 26.65%

26.65134846585898
